# FIFA/Coca-Cola World Ranking
Parsing the FIFA World Ranking from fifa.com.

## First part. 
Get data from 1992 

Columns:
- **id** — counrty id
- **country_full** — country full name
- **country_abrv** — country abbreviation
- **rank** — current country rank
- **total_points** — current total points
- **previous_points** — total points in last rating
- **rank_change** — how rank has changed since the last publication
- **confederation** — FIFA confederations
- **rank_date** — date of rating calculation

In [1]:
import datetime
import asyncio
import aiohttp
import requests as r
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer

# for run asyncio in jupyter / https://github.com/jupyter/notebook/issues/3397
import nest_asyncio
nest_asyncio.apply()

In [2]:
date_id = 'id1'  # first date 31 12 1992
fifa_url = 'https://www.fifa.com/fifa-world-ranking/ranking-table/men/rank'

### Collection of dates when ratings were created

In [3]:
def get_dates_html():
    page_source = r.get(f'{fifa_url}/{date_id}/')
    page_source.raise_for_status()
    dates = BeautifulSoup(page_source.text, 
                          'html.parser', 
                          parse_only=SoupStrainer('li', attrs={'class': 'fi-ranking-schedule__nav__item'}))
    return dates


def create_dates_dataset(html_dates):
    date_ids = [li['data-value'] for li in html_dates]
    dates = [li.text.strip() for li in html_dates]
    dataset = pd.DataFrame(data={'date': dates, 'date_id': date_ids})
    
    # convert 'date' from str to datetime and sorting "old -> new"
    dataset['date'] = pd.to_datetime(dataset['date'], format='%d %B %Y')
    dataset.sort_values('date', ignore_index=True, inplace=True)
    assert dataset.date.min() == dataset.iloc[0].date, \
            "Incorrect dataset sorting"
    
    return dataset    

In [4]:
dates_from_page = get_dates_html()
dates_dataset = create_dates_dataset(dates_from_page)

assert len(dates_from_page) == dates_dataset.shape[0], \
        "Number of dates in html and dataset don't match"

print(f'Last date: {dates_dataset.date.max().date()}')

Last date: 2020-09-17


In [5]:
async def get_rank_page(date_id, session):
    async with session.get(f'{fifa_url}/{date_id}/') as response:
        page = await response.text()
        if response.status == 200:
            return {'page': page, 'id': date_id}
        else:
            print(f'Parse error, page: {response.url}')
            return False
        
        
def scrapy_rank_table(page, date):
    rows = BeautifulSoup(page, 
                          'html.parser', 
                          parse_only=SoupStrainer('tbody')).find_all('tr')
    table = []
    for row in rows:
        table.append({
            'id': int(row['data-team-id']), 
            'country_full': row.find('span', {'class': 'fi-t__nText'}).text, 
            'country_abrv': row.find('span', {'class': 'fi-t__nTri'}).text,
            'rank': int(row.find('td', {'class': 'fi-table__rank'}).text), 
            'total_points': int(row.find('td', {'class': 'fi-table__points'}).text),
            'previous_points': int(row.find('td', {'class': 'fi-table__prevpoints'}).text or 0),
            'rank_change': int(row.find('td', {'class': 'fi-table__rankingmovement'}).text.replace('-', '0')),
            'confederation': row.find('td', {'class': 'fi-table__confederation'}).text.strip('#'),
            'rank_date': date
        })
    return table
    

async def parse_ranks(pages_df):
    fifa_ranking = pd.DataFrame(columns=[
        'id', 'rank', 'country_full', 'country_abrv', 
        'total_points', 'previous_points', 'rank_change', 
        'confederation', 'rank_date'
    ])

    start_time = datetime.datetime.now()
    print("Start parsing.. ", datetime.datetime.now()-start_time)
    
    task_parse = []
    async with aiohttp.ClientSession() as session:
        for date_id in pages_df.date_id.to_list():
            task_parse += [asyncio.create_task(get_rank_page(date_id, session))]
    
        for task in asyncio.as_completed(task_parse):
            page = await task
            if not task:
                continue
            date_ranking = scrapy_rank_table(page['page'], 
                                             pages_df[pages_df.date_id == page['id']].date.iloc[0])
            fifa_ranking = fifa_ranking.append(date_ranking, ignore_index=True)

            if fifa_ranking.rank_date.nunique() % 50 == 0:
                print(f'Complite {fifa_ranking.rank_date.nunique()}/{pages_df.shape[0]} dates')
    
    fifa_ranking.sort_values('rank_date', ignore_index=True, inplace=True)
    print(f'Parsing complite. Time {datetime.datetime.now()-start_time}')
    return fifa_ranking


def data_correction(df):
    """ Handmade """
    # Lebanon has two abbreviations
    df.replace({'country_abrv': 'LIB'}, 'LBN', inplace=True)
    # Montenegro duplicates
    df.drop(df[df.id == 1903356].index, inplace=True)
    # North Macedonia has two full names
    df.replace({'country_full': 'FYR Macedonia'}, 'North Macedonia', inplace=True)
    # Cabo Verde has two full names
    df.replace({'country_full': 'Cape Verde Islands'}, 'Cabo Verde', inplace=True)
    # Saint Vincent and the Grenadines have two full names
    df.replace({'country_full': 'St. Vincent and the Grenadines'}, 'St. Vincent / Grenadines', inplace=True)
    # Swaziland has two full names
    df.replace({'country_full': 'Eswatini'}, 'Swaziland', inplace=True)
    # Curacao transform to Curaçao (with 'ç')
    df.replace({'country_full': 'Curacao'}, 'Curaçao', inplace=True)
    # São Tomé and Príncipe have three full names
    df.replace({'country_full': ['Sao Tome e Principe', 'São Tomé e Príncipe']}, 
               'São Tomé and Príncipe', inplace=True)
    return df


def check_data(ranks_df, dates_df):
    if ranks_df.rank_date.nunique() != dates_df.date.nunique():
        print("Warning! Numbers of rank dates don't match")
    if ranks_df.country_full.nunique() != ranks_df.country_abrv.nunique():
        print("Warning! Number of names and abbreviations does not match")
    if ranks_df.country_full.nunique() != ranks_df.id.nunique():
        print("Warning! Number of names and IDs does not match")
        

def save_as_csv(df):
    df.to_csv(
        f'fifa_ranking-{df.rank_date.max().date()}.csv',
        index=False, 
        encoding='utf-8'
    )
    print('Dataframe saved in currently folder')

In [6]:
fifa_ranking_df = asyncio.run(parse_ranks(dates_dataset))
fifa_ranking_df = data_correction(fifa_ranking_df)
check_data(fifa_ranking_df, dates_dataset)
save_as_csv(fifa_ranking_df)

fifa_ranking_df.head()

Start parsing..  0:00:00
Complite 50/306 dates
Complite 100/306 dates
Complite 150/306 dates
Complite 200/306 dates
Complite 250/306 dates
Complite 300/306 dates
Parsing complite. Time 0:03:27.273658
Dataframe saved in currently folder


,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,43948,1,Germany,GER,57,0,0,UEFA,1992-12-31
1,43873,107,Mozambique,MOZ,9,0,0,CAF,1992-12-31
2,43816,108,Indonesia,IDN,9,0,0,AFC,1992-12-31
3,1882218,109,Antigua and Barbuda,ATG,8,0,0,CONCACAF,1992-12-31
4,43820,110,Jordan,JOR,8,0,0,AFC,1992-12-31
